# Этап 1 | Обучающая выборка | Ранжирование

## 1. Построение обучающей выборки
Пусть имеют место параметризации функций окружающей среды:
$$
\begin{cases}
    E = \sigma_1 (x + D)\\
    S_x = \sigma_2 (x + D)
\end{cases}
-D < x < 0
$$
$$
\begin{cases}
    G = (x + D_0)^2\\
    S_{\tau} = cos(2\pi\tau) + 1
\end{cases}
0 < \tau < 1
$$
Возьмем в качестве $v_i$ стратегии две стадии жизнии зоопланктона: молодых и взрослых особей
$$
x_{J} = A_{J} + B_{J}cos(2\pi\tau)\ \ \ \ \ \ x_{A} = A_{A} + B_{A}cos(2\pi\tau)
$$
Ключевыми макропараметрами являются:

$M_1$ и $М_5$ - количество потребляемой пищи в сутки

$M_2$ и $М_6$ - суточная смертность от хищников

$M_3$ и $М_7$ - метаболическая стоимость вертикальной миграции

$M_4$ и $М_8$ - смертность зоопланктона от неблагоприятных условий среды

        Для молодых особей:
$$
M_1(v_i) = \int_0^1 E(x_{Ji}(\tau))d\tau\ \ \ \ \ \ M_2(v_i) = \int_0^1 S_{\tau}(\tau)S_x(x_{Ji}(\tau))d\tau\ \ \ \ \ \ M_3(v_i) = -\int_0^1 (x'_{Ji}(\tau))d\tau\ \ \ \ \ \ M_4(v_i) = -\int_0^1 G(x_{Ji}(\tau))d\tau
$$
        Для взрослых особей:
$$
M_5(v_i) = \int_0^1 E(x_{Ai}(\tau))d\tau\ \ \ \ \ \ M_6(v_i) = \int_0^1 S_{\tau}(\tau)S_x(x_{Ai}(\tau))d\tau\ \ \ \ \ \ M_7(v_i) = -\int_0^1 (x'_{Ai}(\tau))d\tau\ \ \ \ \ \ M_8(v_i) = -\int_0^1 G(x_{Ai}(\tau))d\tau
$$

Известная функция фитнеса: $J = -s-p-q+\sqrt[2]{4rp+(p+q-s)^2}$

С коэффициентами:

$r_i = \alpha_{A} M_5 + \beta_{A} M_7 + \delta_{A} M_8$


$s_i = \gamma_{A}M_6$


$p_i = \alpha_{J}M_1 + \beta_{J} M_3 + \delta_{J} M_4$


$q_i = \gamma_{J}M_2$

In [249]:
# Эмпирические данные 
alpha_j = 0.0016
alpha_a = 0.006
betta_j = 0.0000007
betta_a = 0.000000075
gamma_j = 0.00008
gamma_a = 0.004
delta_j = 0.000016
delta_a = 0.00006
sigma1 = 1
sigma2 = 1

D = 120
D0 = 70  

Nmax = 100

In [250]:
import random as rn
import pandas as pd
# Генерация параметров для стратегий
# Запись параметров  
def rand_AB(Nmax):
   A_j, B_j, A_a, B_a = [], [], [], []
   for i in range(0, Nmax):
      Aj = round(rn.uniform(-D, 0), 4)
      Bj = round(rn.uniform(-min(Aj + D, -Aj), min(Aj + D, -Aj)), 4)
      A_j.append(Aj)
      B_j.append(Bj)

      Aa = round(rn.uniform(-D, 0), 4)
      Ba = round(rn.uniform(-min(Aa + D, -Aa), min(Aa + D, -Aa)), 4)
      A_a.append(Aa)
      B_a.append(Ba)
   
   data_strat = {'A_J': A_j, 'B_J': B_j, 'A_A': A_a, 'B_A': B_a}
   data = pd.DataFrame(data = data_strat)
   data.to_csv("data_strat.csv")
   return data
print("Выборка стратегий поведения создана")
data_strat = rand_AB(Nmax)
data_strat

Выборка стратегий поведения создана


,A_J,B_J,A_A,B_A
0,-2.6924,-0.5210,-89.3342,17.8657
1,-44.8680,18.5028,-25.1766,0.4493
2,-114.3107,1.7159,-35.5249,-34.0253
3,-23.2251,-1.3684,-38.2439,22.1683
4,-13.7898,-4.9783,-70.7592,-34.3670
...,...,...,...,...
95,-119.8097,0.0164,-46.6132,21.7917
96,-44.0704,10.8384,-58.9411,51.4057
97,-78.4500,-3.0962,-52.2976,45.3284
98,-2.8396,-2.7411,-38.8360,-27.2803


In [252]:
import pandas as pd
import numpy as np
# Подсчет макропараметров
def macroparams(Nmax):
   data_strat = pd.read_csv('data_strat.csv')
   Aj = data_strat['A_J'].tolist()
   Bj = data_strat['B_J'].tolist()
   Aa = data_strat['A_A'].tolist()
   Ba = data_strat['B_A'].tolist()
   M1, M2, M3, M4, M5, M6, M7, M8 = [], [], [], [], [], [], [], []
   for i in range(0, Nmax):
      M1.append(round(sigma1*(Aj[i] + D), 5))
      M2.append(round(-sigma2*(Aj[i] + D + Bj[i]/2), 5))
      M3.append(round(-2*(np.pi*Bj[i])**2, 5))
      M4.append(round(-((Aj[i] + D0)**2 + (Bj[i]**2)/2), 5))
      M5.append(round(sigma1*(Aa[i] + D), 5))
      M6.append(round(-sigma2*(Aa[i] + D + Ba[i]/2), 5))
      M7.append(round(-2*(np.pi*Ba[i])**2, 5))
      M8.append(round(-((Aa[i] + D0)**2 + (Ba[i]**2)/2), 5))

   data_macro = {'M1': M1, 'M2': M2, 'M3': M3, 'M4': M4, 'M5': M5, 'M6': M6, 'M7': M7, 'M8': M8}
   data = pd.DataFrame(data = data_macro)
   data.to_csv("data_macro.csv")
   return data
print('Подсчитаны макропараметры')
data_macro = macroparams(Nmax)
data_macro

Подсчитаны макропараметры


,M1,M2,M3,M4,M5,M6,M7,M8
0,117.3076,-117.04710,-5.35803,-4530.44874,30.6658,-39.59865,-6300.42455,-533.40291
1,75.1320,-84.38340,-6757.78935,-802.79423,94.8234,-95.04805,-3.98476,-2009.23812
2,5.6893,-6.54725,-58.11841,-1964.91029,84.4751,-67.46245,-22852.49735,-1767.39304
3,96.7749,-96.09070,-36.96203,-2188.82753,81.7561,-92.84025,-9700.50896,-1254.16665
4,106.2102,-103.72105,-489.20611,-3171.97832,49.2408,-32.05730,-23313.79572,-591.12173
...,...,...,...,...,...,...,...,...
95,0.1903,-0.19850,-0.00531,-2481.00635,73.3868,-84.28265,-9373.71973,-784.38151
96,75.9296,-81.34880,-2318.78291,-731.07961,61.0589,-86.76175,-52161.76712,-1443.57227
97,41.5500,-40.00190,-189.22903,-76.19573,67.7024,-90.36660,-40557.43869,-1340.70689
98,117.1604,-115.78985,-148.31310,-4514.27614,81.1640,-67.52385,-14690.21070,-1343.30228


In [287]:
import pandas as pd
# Счатем фитнесс
def fitness(Nmax):
   data_macro = pd.read_csv('data_macro.csv')
   M1 = data_macro['M1'].tolist()
   M2 = data_macro['M2'].tolist()
   M3 = data_macro['M3'].tolist()
   M4 = data_macro['M4'].tolist()
   M5 = data_macro['M5'].tolist()
   M6 = data_macro['M6'].tolist()
   M7 = data_macro['M7'].tolist()
   M8 = data_macro['M8'].tolist()
   J = []
   r, s, p, q = 0, 0, 0, 0
   for i in range(0, Nmax):
      r = alpha_a*M5[i] + betta_a*M7[i] + delta_a*M8[i]
      s = gamma_a*M6[i]
      p = alpha_j*M1[i] + betta_j*M3[i] + delta_j*M4[i]
      q = gamma_j*M2[i]
      root = 4*r*p + (p + q - s)**2
      if root < 0:
         J.append(0)
      else:
         j = -s - p - q + np.sqrt(root)
         J.append(j)
   return J
data_J = fitness(Nmax)
data_J


[0.42623997864320967,
 0.9251906184734568,
 0.4516937020049475,
 0.9168150091656808,
 0.4416739456447679,
 0.1419545640942583,
 0.5473692700322388,
 0.890036401160474,
 0.9703202958353585,
 0.07389028277696869,
 0.8623568927640746,
 1.0166582954100758,
 0.9110993666625494,
 0.8504817768822168,
 0.613457329659653,
 0.50128954277217,
 0.48558066084964846,
 0.6445945738601627,
 1.0895250818312083,
 0.06460245034424619,
 0.6868068578888733,
 0.6603787495039599,
 0.015754539232467134,
 0.5924616691451032,
 0.4562201541725998,
 0.812189682791326,
 0.9327164065439975,
 1.042082214559916,
 0.9483444373092719,
 0.9791017572387215,
 0.6111777571326473,
 0.7878167642078056,
 0.8074274549284861,
 1.0393509237794334,
 0.2704467986860083,
 0,
 0.7261457472937092,
 1.0175495992316845,
 0.9895762379954884,
 0.42484911062399583,
 0.9198497377704931,
 1.0553397129560955,
 0.9269592242158895,
 0.1887388417868064,
 0.3363674361040905,
 0.39197074961724476,
 0.44505942080349875,
 0.757643341962256,
 0.3685

$$----------------------------------------------------------------------$$